In [5]:
import re
import pandas as pd
import plotly.express as px

from glob import glob

In [6]:
pt_time = re.compile(r"\[epoch (\d)\] elapsed time\: (\d+\.\d+) sec")

In [7]:
logs = sorted(glob("./**/**/torch.log"))
rows = []

for log in logs:
    with open(log, "r") as f:
        lines = f.readlines()

    time_per_epoch = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    for line in lines:
        sh_time = pt_time.search(line)
        if sh_time:
            time_per_epoch[int(sh_time.group(1))] = float(sh_time.group(2))

    rows.append(
        {
            "model": log.split("/")[1],
            "gpus": "H100 * 8",
            "storage": log.split("-")[-1].split("/")[0],
            "epoch_1": round(time_per_epoch[1], 2),
            "epoch_2": round(time_per_epoch[2], 2),
            "epoch_3": round(time_per_epoch[3], 2),
            "epoch_4": round(time_per_epoch[4], 2),
            "epoch_5": round(time_per_epoch[5], 2),
        }
    )

In [13]:
df = pd.DataFrame(rows)
df.sort_values(by=["model", "gpus"], ascending=[False, True], inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns="epoch_1", inplace=True)
df["mean"] = round(df.iloc[:, [3, 4, 5, 6]].mean(axis=1), 2)
df["standard_deviation"] = round(df.iloc[:, [3, 4, 5, 6]].std(axis=1), 2)
df.to_csv("data_storage.csv", index=False)
df

,model,gpus,storage,epoch_2,epoch_3,epoch_4,epoch_5,mean,standard_deviation
0,Llama-2-13b-chat-hf,H100 * 8,gpfs,31.16,31.14,31.04,31.05,31.10,0.06
1,Llama-2-13b-chat-hf,H100 * 8,ontap,31.09,31.10,30.98,31.03,31.05,0.06
